# ENGR 76 Project 2b: Filtering and Frequency-division multiplexing

_Due: May 30, 2025 at 16:00 PDT_

<font color='blue'>**YOUR STANFORD EMAIL:**</font>     @stanford.edu

If your transmission is the only sound in the room, then the receiver can just listen to it and decode it to recover the message, as you did in Project 2a. But in reality, we never have such a happy monopoly on the transmission environment. Rather, the receiver will have to deal with other sounds, like other background noise, conversations or other transmitters. Any sound that isn't the transmitted signal is called _noise_. This week, we'll explore how filtering can be used to remove noise and extract the signal.

<div class="alert alert-info">
    Wireless communication, including 4G, Wi-Fi, Bluetooth, FM radio and GPS signals, has the same problem—both from other wireless transmitters and from other sources of electromagnetic energy. The techniques they use to extract their signal of interest are similar.
</div>

In this project, you will:

- Establish how to work with the FFT as a practical approximation for a signal's spectrum
- Inspect the spectra of your transmitted OOK signal, and your received one, with and without noise
- Apply filtering to the received noisy signal and show that filtering improves the bit error rate
- Investigate how different frequency bands can be used for independent transmissions
- Implement frequency multiplexing to increase the data rate of your system
- Consider how frequency multiplexing can be used to allow several systems to transmit in the same space

<div class="alert alert-info" style="color:black;">
    <p>
        <strong>Where we are in Project 2.</strong>
    </p>
    <p style="text-align: center;">
        <img src="img/proj2b-where.png" style="height: 84px;" />
    </p>
</div>

<div class="alert alert-info" style="color: black;"><strong>Answer formatting:</strong> As usual, we've left <code>&gt; YOUR ANSWER HERE</code> in cells where we want you to type in answer. To do so, double-click the cell, and please leave the <code>&gt;</code> there (and start each paragraph with <code>&gt;</code>) so that the blockquote bar on the left remains next to your answer.</div>

<div class="alert alert-warning">Throughout Project 2, you may reuse some recorded signals across different code cells. To prevent loading incorrect signals, <b>pay additional attention to your variable names to ensure they are not accidentally overwritten.</b></div>


In [ ]:
# Packages you should already have installed, to be imported:

import numpy as np

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 5)  # make the graphs wider

from numpy.random import default_rng

rng = default_rng()

# setting up the channels
save_wav = False #SET TO TRUE IF AUDIO IS NOT WORKING
from channel import Channel, save
from functools import partial

channel = Channel()
channel0 = partial(channel.transmit, save_wav=save_wav)
channel1 = partial(channel.transmit, talking=True, save_wav=save_wav)
channel2 = partial(channel.transmit, clap=True, save_wav=save_wav)

import IPython.display as ipd

# Part 1: Getting to know the FFT (10 points)

The _fast Fourier transform_ (FFT) is an algorithm for the _discrete Fourier transform_ (DFT), which is a version of the Fourier transform suitable for discrete-time, finite-length signal. We use FFT throughout this project to obtain the spectrum of time-domain signals.

Technically, the FFT works on discrete-time signals,
$$x[n], \quad n = 0, \dots, L-1,$$
and returns a frequency-domain representation that is one more than half the length. We can call this $x_\mathcal F$:
$$x_\mathcal{F}[n], \quad n = 0, \dots, \lfloor L/2 \rfloor.$$

However, we often want to use the FFT as an **approximation** to the continuous-time Fourier transform. For example, say that $x[n]$ was a sampled version of a continuous-time signal $\tilde x(t)$ at sampling frequency $f_s$ ,

$$x[n] = \tilde x(t[n]), \qquad t[n] = \frac{n}{f_s}.$$

Recall from last week that to plot this signal using `plt.plot()` in a way that makes sense for “the real world”, we plot the time vector $t[n]$ against the signal vector $x[n]$. Similarly, when working with the FFT $x_\mathcal{F}[n]$, we'll want to plot it against some sort of “frequency vector”
$$f[n], \quad n = 0, \dots, \lfloor L/2 \rfloor.$$

Our goal in this part is to establish how to think about what $f[n]$ should be.

### FFT of a pure tone

Let's start with a **pure tone**.

In [ ]:
# feel free to play with these numbers and then re-run subsequent cells
tmax = 0.1  # end of signal in "real" time (seconds)
fs = 44100  # sampling frequency
f = 1000  # frequency of tone
t = np.arange(0, tmax, 1 / fs)  # time vector t[n]
x = np.sin(2 * np.pi * f * t)  # signal vector x[n]

# plot signal in time domain
plt.plot(t, x)
plt.xlabel('Continuous-time variable, t', size=18)
plt.ylabel('Continuous x(t)', size=18)

We can take the FFT using the `rfft()` function, which is in the the `scipy.fft` module.

In [ ]:
# import relevant functions
from scipy.fft import rfft, irfft, rfftfreq

If you inspect the spectrum `rfft(x)`, you'll notice that its entries are complex. In class we had the convention of having separate $a_j$, $b_j$ coefficients for sine and cosine. In the projects we will combine them into a single complex value $x_\mathcal{F}[j] = b_j + ia_j$. For $j=0$, the value is guaranteed to be real as we only have $b_0$. To visualize it effectively as a plot, it's common to take its absolute value (also known as its modulus). That is, instead of plotting $x_\mathcal{F}[n]$ (called `x_fft` below), we instead plot $|x_\mathcal{F}[n]|$.

In [ ]:
x_fft = rfft(x)
print(x_fft.dtype)
plt.plot(np.abs(x_fft))
plt.ylabel('Amplitude', size=18)

One problem: The above code plots with respect to frequency "indices" $n$, which aren't very meaningful here. That spike should be at whatever `f` was set to above.

<div class="alert alert-warning">This is a key distinction between our work in this project, and our study of the DCT for images in Project 1b. In Project 1b, the “duration” of the signal (<i>i.e.</i> size of the image) had no physical meaning, so we ignored the units. Here, time is <em>measured in seconds</em> and frequency is <em>expressed in Hertz</em> (the reciprocal of seconds), so we have to be more precise with our handling of frequencies.</div>

Whenever we need to plot the FFT of a signal, we also need to generate a vector of frequencies, so that the horizontal axis is scaled appropriately. To establish the scaling, note the following:
- The number of elements in the FFT output is half the length of the original signal $L$
- The highest frequency that the original signal can capture is half the sampling frequency $f_s$

The frequency vector is therefore given by

$$f[n] = \frac{f_s/2}{L/2} \cdot n = \frac{f_s}{L} \cdot n = \frac{1}{L T_s} \cdot n, \qquad \text{for } n = 0, \dots, \lfloor L/2 \rfloor.$$

where $L$ is the length of the signal vector $x[n]$, $f_s$ is the sampling frequency, and $T_s = 1/f_s$ is the sampling period.

NumPy provides a convenience function that does this for us. It's called `rfftfreq`, and it takes two arguments: the length of the original signal $L$, and the sampling period $T_s$.

In [ ]:
Ts = 1 / fs
f_fft = rfftfreq(x.size, Ts)  # equivalent to: np.arange(x_fft.size) * fs / x.size
x_fft = rfft(x)

plt.plot(f_fft, np.abs(x_fft))
plt.xlabel("Frequency (Hz)", size=18)
plt.ylabel("Amplitude", size=18);

So to summarize, to use the FFT as an approximation for the continuous-time Fourier transform, we do what's in the above cell.

<div class="alert alert-info">
    <p><strong>More than you needed to know:</strong> The <code>r</code> in <code>rfft()</code> stands for “real”, signalling that <code>rfft()</code> computes the FFT on the assumption that the signal is real (<i>i.e.</i>, without imaginary part), which ours is. The frequency-domain representation (FFT output), however, is still complex (<i>i.e.</i>, has an imaginary part).</p>
    <p>Then what does the real assumption buy us? The standard <code>fft()</code> computes the FFT assuming that the input might be complex. This gives rise to “negative frequencies”, which are beyond the scope of this course. However, if the input is real, then the negative frequencies form a symmetry with the positive frequencies. This renders the negative frequencies redundant, so <code>rfft()</code> just doesn't compute them.</p>
    <p>The frequency-domain representation is complex to account for both the amplitude and phase of frequency components. The magnitude (a.k.a. absolute value, modulus) repreesnts the amplitude, and the argument (a.k.a. angle) represents the phase. In this project, we will only be interested in inspecting the magnitude of frequency components. However, it's necessary to retain the whole complex number to be able to do the inverse FFT, which we'll do next.</p>
</div>

### Inverse FFT

The function computing the inverse FFT is `irfft()`.

Like `rfft()`, `irfft()` doesn't itself know anything, or need to know anything, about sampling. It just carries out the inverse FFT algorithm on frequency components, presumably indexed by integers. The corresponding time vector $t[n]$ is then the same time vector that we started with, $t[n] = nT_s$.

In [ ]:
x_recovered = irfft(x_fft)
# just use the same `t` from before
plt.plot(t, x_recovered)
plt.xlabel('t', size=18)
plt.ylabel('Recovered x(t)', size=18)

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 1.1.</span> Choose two other frequencies, and make a signal with exactly those two frequency components (by adding two pure tones together). Plot the FFT vector $x_\mathcal{F}[n]$ against the correct frequency vector $f[n]$, such that the spikes are in the right place. Plot the result of the inverse FFT to verify it recovers the original signal.

In [ ]:
# Cell for Question 1.1
# plot frequency spectrum
f_fft = None
x_fft = None
##### YOUR CODE HERE ######
###########################

In [ ]:
# Cell for Question 1.1
# plot time domain signal
##### YOUR CODE HERE ######
###########################

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 1.2.</span> Is the spectrum plot what you expected? How many peaks do you see, and at what frequencies?

> YOUR ANSWER HERE

# Part 2: The spectrum of the transmitted signal (20 points)

We're now in a position to begin our investigation of the spectrum of our transmitted OOK signal. Let's start with the **baseband** (unmodulated) signal.

<div class="alert alert-info"><strong>Terminology:</strong> In this project, the <em>baseband signal</em>, <em>unmodulated signal</em> and <em>envelope signal</em> are all the same thing. Similarly, <em>passband signal</em> and <em>modulated signal</em> refer to the same thing. (In other contexts, there might be subtle differences between these terms, but in this project, there aren't.)</div>

### Spectrum of baseband (unmodulated, envelope) signal

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 2.1.</span> Use your code from Project 2a to generate a **baseband** (envelope) signal for a randomly generated message. 

**IMPORTANT NOTE**: Use the `rfft()` and `rfftfreq()` functions to plot the magnitude (`np.abs()`) of its spectrum. This will be required for all subsequent questions where we ask you to plot the spectrum of a signal.

Use a data rate of 300 bits/second. Your message should be of a length that the entire transmission takes around 6 seconds.

**Zooming in on plots:** If you want to zoom in horizontally on the plot, you can use the command <code>plt.xlim(xmin, xmax)</code> to specify the limits of the x-axis. For example, `plt.xlim(0, 10)` will restrict the plot to between 0 and 10 on the x-axis. This can allow you to zoom in on the parts of the spectrum that are more interesting. Similarly, you can use <code>plt.ylim(ymin, ymax)</code> to zoom in on the y-axis (vertically). This might allow you to more closely inspect the nonzero parts of the spectrum that aren't the big main spike.

To get a better understanding of the spectrum, you will require to zoom in horizontally. Please present **both** the original plot **and** the zoomed in plot. (It's okay to duplicate plotting code between cells to make this work.)


In [ ]:
# Question 2.1.
# Spectrum of the baseband (envelope) signal.
rate = 300
message = rng.integers(0, 2, 1800)
message = np.insert(message, 0, 1)  # always start the message with a 1
baseband = None #replace this
# As always, add as many cells as you need to show what you want to show.

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 2.2.</span> Examine the spectrum plot (with appropriate zooming-in) for the baseband signal and comment on the shape of the spectrum. Is this similar to what we learnt in class? At what frequency, does the spectrum first become zero? How is this related to the data rate we set above? 

> YOUR ANSWER HERE

### Spectrum of modulated (passband) signal

Now modulate the signal and inspect its spectrum.

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 2.3.</span> Use your code from Project 2a to **modulate** your baseband signal at a carrier frequency of 1800 Hz to arrive at the **modulated (passband) signal** (_i.e._, multiply by a pure tone at your the carrier frequency), and plot the spectrum of the modulated signal.

<div class="alert alert-success" style="color: black;"><strong>General notes on plots.</strong> In all spectrum plots in this project, you should:
    <ul>
        <li>plot only the magnitude of the spectrum, <i>i.e.</i> take the <code>np.abs()</code> of the (complex) spectrum before plotting, and</li>
        <li>feel at liberty to zoom in to the plot using <code>plt.xlim()</code> (and <code>plt.ylim()</code> if you like), but please always present the original zoomed out plot as well, so we can see what you're zooming into.</li>
    </ul>
</div>

In [ ]:
# Question 2.3.
fc = 1800
# Spectrum of passband signal.
# YOUR CODE HERE
# As always, add as many cells as you need to show what you want to show.

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 2.4.</span> Examine the spectrum plot (with appropriate zooming-in) for the modulated signal and comment on the shape of the spectrum. 
- Is this similar to what we learnt in class? What frequency is the spectrum centered at (i.e., the frequency at which the amplitude of the spectrum is the largest)?
- In class, we defined the approximate band the signal lies in as the the region between the first zero corssings on either side of the peak. What is the approximate band the signal lies in? Does this match our choice of rate above?

> YOUR ANSWER HERE

# Part 3: The spectrum of the received signal (20 points)

It's useful to understand the spectrum of the transmitted signal because it gives us an idea of what to expect from a "perfect received signal" (which is, in an ideal world, just a scaled version of the transmitted signal). In this part, we'll inspect the spectrum of the received signal.

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 3.1.</span> We have given you the code to transmit and receive your modulated signal (using `channel0()`). Recall that we add silence before passing the signal through the channel to allow the whole signal to be captured by the receiver.

Please plot the spectrum of the received signal. 

_Note:_ The silence is added after the signal to make sure the entire OOK signal is recorded. However, you don't need to truncate the signal to detect its starting point accurately—just take the FFT of the whole thing. Note that this means your frequency vector $f[n]$ (generated by `rfftfreq()`) will be different to the one you used to plot the spectrum in Part 2, as the signal is longer now.

<div class="alert alert-info">Reminder: You can zoom in using <code>plt.xlim()</code> and <code>plt.ylim()</code>, which you might do to look more closely at the “interesting” parts of the spectrum. It's okay to zoom in vertically in a way that pushes the main spike off the scale, if you want to look at other activity.</div>

In [ ]:
# Now that we don't need to separately use the baseband and modulated signals,
# you can directly use the functions from 2a.
# Feel free to replace with your answers from 2a.
def add_silence(signal: np.ndarray) -> np.ndarray:
    """ Adds an appropriate amount of silence to the signal.
        - signal: 1-D array of floats
    Output variables:
        - new_signal: 1-D array of floats
    """
    silence = 1  # seconds
    new_signal = np.append(signal, np.zeros(int(silence * fs)))
    return new_signal


def generate_modulated_signal(message: np.ndarray, rate: float, fc: float) -> np.ndarray:
    """ Generate the modulated signal.
    Input variables:
        - message: 1-D array of ints
        - rate:    data rate in bits per second
        - fc:      carrier frequency in Hz
    Output variables:
        - signal: modulated signal as 1-D array of floats
    """
    T = 1 / rate
    k = len(message)
    t = np.arange(int(k*T*fs))/fs
    envelope = message[np.floor(t / T).astype(int)]
    signal = np.sin(2 * np.pi * fc * t) * envelope
    return signal

In [ ]:
# Question 3.1
# Spectrum of the received signal.
rate = 300
fc = 1800
message = rng.integers(0, 2, 1800)
message = np.insert(message, 0, 1)  # always start the message with a 1

x = generate_modulated_signal(message, rate, fc)
x_extended = add_silence(x)
y = channel0(x_extended)

##### YOUR CODE HERE ######
###########################


<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 3.2.</span> Comment on how this is similar to and different from the spectrum of the transmitted passband signal you found in Question 2.3. For example—is the spike in the same place, are there additional components or absent components?

_WHAT WE EXPECT_: a short answer with 1-2 sentences.

> YOUR ANSWER HERE

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 3.3.</span> The following cell records noise from `channel1` (the one while talking) for 6 seconds.

You should first plot the talking noise (in the time domain). This will help you understand how an audio of someone talking looks like before. Then plot the spectrum of talking noise in the following cell.

In [ ]:
# You don't have to change anything here. You can just run the cell.
duration = 6
noise = channel1(np.zeros(duration * fs))

In [ ]:
# Question 3.3 
# plot the signal in the time domain 

In [ ]:
# Question 3.3.
# Spectrum of the "talking noise" recording.

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 3.4.</span> We are transmitting the  signal through `channel1`.
Plot the spectrum of your received signal in the following cell.

What is the relationship between this spectrum and the ones in Questions 3.3 and 2.3?

_WHAT WE EXPECT_: a short answer with 1-2 sentences.

> YOUR ANSWER HERE

<div class="alert alert-warning">You may reuse the received signal in the subsequent questions, so to prevent error, <b> we suggest you to use a unique variable to store it </b> (e.g.,  <code>y_talking</code>).</div>

In [ ]:
### Question 3.4
y_talking = channel1(x_extended)


###### YOUR CODE HERE ##########

################################

# Part 4: Filtering to extract the signal of interest (35 points)

Recall from Project 1b that a **filter** is an operation that extracts some frequency components, and sets all others to zero.

In this case, we want to keep the frequencies that are likely to be relevant to our OOK signal, and to discard the ones that aren't. This isn't necessarily a clean-cut decision. You'll have noticed in Part 3 that there's some overlap involved. Nonetheless, we can "clean up" our signal quite a lot by choosing cutoff frequencies strategically.

Because our frequencies have physical meanings, our filter implementation will be a little more involved than the filter from Project 1b. Remember the scaling for frequencies we had to do so that we can plot spectra with accurate frequency axes? We'll need to do that here, too. So we'll first write a function that modifies a spectrum by retaining _only_ those frequencies within a _passband_, between `fmin` and `fmax`.

(On the other hand, we only have to deal with one dimension in this one!)


<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.1.</span> Write a function `bandpass()` that takes a spectrum (as returned by `rfft()`) and sampling frequency `fs`, and returns the same spectrum with all values outside the frequency range between `fmin` and `fmax` set to zero.

- The tricky part of this function is that you'll need to “convert” `fmin` and `fmax` to indices of the `spectrum` vector. To do this, recall from earlier that
  $$f[n] = \frac{f_s}{L} \cdot n,$$
  where $f_s$ is the sampling frequency and $L$ is the length of the original vector. Note that $L$ is **twice the length of the spectrum**.
- Your function should not need to use `rfft()` or `irfft()`, because it is only working in the frequency domain. In subsequent cells, call these functions separately to `bandpass()`.
- If you use `np.zeros()` (or a similar function) to initialize a new array, add the keyword argument `dtype=complex`. This will make sure the array expects to take complex numbers, since the spectrum is complex.
- You shouldn't need to modify any of the complex numbers in the array, other than to replace them with zero.
- Note that your filter can either include or exclude $f_\text{min}$ and $f_\text{max}$. We will not be strict about this.

In [ ]:
def bandpass(spectrum, fs, fmin, fmax):
    # replace this line
    return np.zeros(spectrum.size, dtype=complex)

You can check your implementation using the below cell.
After the bandpass, the spectrum should look like the original between 5000 and 10,000 Hz, and zero everywhere else.
Note that the input to `bandpass` is complex-valued!

In [ ]:
# Just run this cell and look at the plots
test_freqs = rfftfreq(1000, 1 / fs)
test_og_fft = 7000**2/(-test_freqs**2 + 7000j*test_freqs/3 + 7000**2)
test_bandpass_fft = bandpass(test_og_fft, fs, 5000, 10000)
fig0 = plt.figure()
plt.plot(test_freqs, np.abs(test_og_fft))
plt.xlabel('frequency (Hz)')
plt.title("Before bandpass")
fig1 = plt.figure()
plt.plot(test_freqs, np.abs(test_bandpass_fft))
plt.title("After bandpass")
plt.xlabel('frequency (Hz)')

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.2</span> Apply the bandpass filter to the spectrum of your noisy signal `y` from Question 3.1. Recall that this was the output from `channel0`. Plot the resulting filtered signal in both the time domain and the frequency domain. You should use this plot to verify that your `bandpass()` implementation works as intended.

You'll need to choose the cutoff frequencies `fmin` and `fmax`. You should choose them based on what you notice about the spectra in Part 3, and the definition of the approximate band we learnt in class.

<div class="alert alert-warning">Note that the noisy signal in Question 3.1 is free from "interference," such as human talking noise introduced in Question 3.4, so filtering is unnecessary for decoding. You can use a simple energy-based decoder as in Project 2a. The intention of this question is primarily to serve as a sanity check to your filtering implementation. However, in the following questions where the received signal is affected by human voice contamination, filtering should enhance your decoding. </div>

In [ ]:
# Question 4.2.
# Plots of filtered signal in time and frequency domain.
y_filtered = None #name for the filtered time domain signal

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.3</span> If you haven't already, zoom in to a section of the time-domain signal that's about 10 to 15 symbols long. Plot both the unfiltered received signal, and the filtered signal (on different axes), in the same zoomed-in section.
(If you did this during Question 4.2., just leave a note in the answer below telling us to look above.)

In [ ]:
# Question 4.3
# Plots of unfiltered and filtered signals.

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.4</span> Listen to the original received signal `y` and the filtered received signal `y_filtered`. What sounds different?

_WHAT WE EXPECT_: a short answer with 1-2 sentences.

> YOUR ANSWER HERE

In [ ]:
ipd.Audio(y, rate=fs)

In [ ]:
ipd.Audio(y_filtered, rate=fs)

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.5</span> How did you end up choosing your filter cutoff frequencies? Explain your thinking.

_WHAT WE EXPECT:_ a short answer with 1-2 sentences.

> YOUR ANSWER HERE

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.6</span> Decode the _filtered_ signal using the same techniques that you used in Project 2a. Report the bit error rate. We have given you the required helper functions; you may replace them with your own implementation from Project 2a.

<div class="alert alert-success"><strong>General note:</strong> You might find it helpful to plot the received signal and visualize the energy values, possibly after zooming in. This can help you find any bugs in the code, or to set the thresholds at an appropriate level. You should check this whenever you make any major changes to the parameters, or when you can't properly understand why you are getting bad performance.</div>

In [ ]:
def visualize(signal: np.ndarray, start_threshold, start_idx, end_idx) -> None:
    """ Gives a visualization of where your threshold and given indices are in relation to the signal.
    You can use this to debug how you detect your first sample, or to see if you have the right samples corresponding to each bit.
    """
    plt.plot(signal, label="signal")
    if start_threshold is not None:
        plt.axhline(start_threshold, ls='--', label="start threshold", c='C1')
    if start_idx is not None:
        plt.axvline(start_idx, label="start", c='C2')
    if end_idx is not None:
        plt.axvline(end_idx, label="end", c='C2')
    plt.legend()
    
def detect_start(signal, visual_test=False):
    # you should define start_threshold and start_index as explained above
    absolute_values = np.abs(signal)
    rms = np.sqrt(np.mean(signal ** 2))
    start_threshold = rms*np.sqrt(2)
    start_index = np.nonzero(absolute_values > start_threshold)[0][0]

    if visual_test:
        visualize(signal, start_threshold, start_index, None)
    return start_index


def demodulate(signal: np.ndarray, rate: float, length: int) -> np.ndarray:
    """ Demodulate the receieved signal
    Input variables:
        - signal:  1-D array of ints
        - rate:    data rate in bits per second
        - length   expected length of message
    Output variables:
        - decoded: modulated signal as 1-D array of floats
    """
    start_index = detect_start(signal, visual_test=False)
    bit_threshold = np.mean(signal[start_index:] ** 2) / 2

    decoded = np.zeros(length, dtype = np.int8)
    for i in range(length):
        start = (i * fs) // rate + start_index
        end = ((i + 1) * fs) // rate + start_index
        power = np.mean(signal[int(start): int(end)] ** 2)
        decoded[i] = 1 if (power > bit_threshold) else 0

    return decoded

In [ ]:
#### Code for Question 4.6

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.7</span> Filter `y_talking`, which was the output of `channel1` in Question 3.4. Listen to the original and filtered received signals. Describe how the signal has changed.

_WHAT WE EXPECT_: a short answer with 1-2 sentences.

> YOUR ANSWER HERE

In [ ]:
##### YOUR CODE HERE ######
###########################
ipd.Audio(y_talking_filtered, rate=fs)

In [ ]:
ipd.Audio(y_talking, rate=fs)

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question 4.8</span> Decode the message using the filtered version of `y_talking`. Report the bit error rate. There should be a significant improvement over the results for channel1 in project 2a, and at most 0.05 (5%). If you are getting very high error rates (~0.5), you may want to rerun Question 3.4, and generate a new `y_talking`.

In [ ]:
##### YOUR CODE HERE ######

# Analysis (5 points)

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question A1.</span> For what kinds of noise or interference would the filtering strategy we implemented in this project _not_ be effective? What sort of coordination might be necessary if there are multiple transmitters operating in the same space?

_WHAT WE EXPECT:_ a paragraph explaining your thoughts.

> YOUR ANSWER HERE

# Reflection (5 points)

<span style="background-color: #8C1515; font-weight: bold; color: white;">Question R1.</span> Here it is: https://forms.gle/wcizU7zJp3CLqvQC7

# <font color="blue"> Submission guidelines</font>

## Export this Jupyter notebook as a PDF file

- After finishing all the questions, make sure they display properly.
- Save this notebook as a pdf file. Feel free to use any method for exporting the PDF, provided that the resulting document remains readable. Below are a few options:

    1. We suggest the simplest approach: save the notebook as an HTML file, open it in a web browser, then right-click the page and select "Print" to generate a PDF.
    2. Another way to convert your .ipynb files to .pdf is to upload the .ipynb file to Google Drive, and then double-clicking the file to open it in Google Colaboratory. You should see that all of your code and code outputs are there. **Please be sure to not run any cells on Colab, as this will overwrite your output!** You can then click File > Print, and then select Save as PDF in the print dialog.
    3. You can also convert your notebook in the following websites:
        - https://onlineconvertfree.com/convert/ipynb/
        - https://2pdf.com/convert-ipynb-to-pdf/

## What and where to submit

Submit this Jupyter notebook  and the PDF to Gradescope. There are two portals, one for the PDF and the other for the notebook. We won't use the autograder this time, so the `.ipynb` notebook will only be used when we need to verify the correctness of your code.


- Submit this Jupyter notebook file (as a .ipynb file) to the "**Project 2b (jupyter-notebook)**" assignment on Gradescope.
- Submit the PDF to the "**Project 2b (PDF)**"
- Don't forget to fill out the reflection form!


## Coding style and formatting (5 points)

- Coding style: This isn't a development project, so we don't plan to be very particular about this. But we still have to read your code, so please try to make that reasonably painless for us. For example, please remove commented-out code and redundant variables before you submit.

- Formatting: In addition to coding style, please make sure that your exported PDF is readable (for example, your submission shouldn't be a long, single-paged PDF file) and that all the questions are properly tagged. Hopefully these 5 points will be effectively free points.